In [1]:
from tkgngc.embeddings import PretrainedTKGEmbeddingWithTimestamps
from tkgngc.model import NGCWithPretrainedTKGAndTimestamps

In [2]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature_eng.scalers import ranged_scaler
from datetime import datetime, timedelta
from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 

In [3]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

In [4]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

In [5]:
for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [6]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)
cats_rows_list = metadata.rows(named=True)

In [7]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()


timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.142857,-0.5,-4.1078e-14,2.0428e-14,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180547,-0.507953,-0.716059,-0.774361,0.100389,-0.186623,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.142857,-0.495998,0.00002,0.0002,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.18054,-0.507953,-0.716059,-0.774361,0.100389,-0.186618,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.142857,-0.486172,0.00004,0.0004,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180519,-0.507953,-0.716059,-0.774361,0.10039,-0.186604,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.142857,-0.463453,0.00006,0.0006,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180484,-0.507953,-0.716059,-0.774361,0.100391,-0.18658,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.142857,-0.444095,0.00008,0.0008,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180437,-0.507953,-0.716059,-0.774361,0.100393,-0.186548,0.0,0.0,4


In [8]:

cats_df = cats_df.to_pandas()

In [9]:
cats_df.head()

,timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,...,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
0,2023-01-01 00:00:00,0.0,0.142857,-0.500000,-4.107825e-14,2.042810e-14,0.0,0.0,-0.32802,-0.369237,...,-0.767181,-0.180547,-0.507953,-0.716059,-0.774361,0.100389,-0.186623,0.0,0.0,0
1,2023-01-01 00:00:01,0.0,0.142857,-0.495998,2.000000e-05,2.000000e-04,0.0,0.0,-0.32802,-0.369237,...,-0.767181,-0.180540,-0.507953,-0.716059,-0.774361,0.100389,-0.186618,0.0,0.0,1
2,2023-01-01 00:00:02,0.0,0.142857,-0.486172,4.000000e-05,4.000000e-04,0.0,0.0,-0.32802,-0.369237,...,-0.767181,-0.180519,-0.507953,-0.716059,-0.774361,0.100390,-0.186604,0.0,0.0,2
3,2023-01-01 00:00:03,0.0,0.142857,-0.463453,6.000000e-05,6.000000e-04,0.0,0.0,-0.32802,-0.369237,...,-0.767181,-0.180484,-0.507953,-0.716059,-0.774361,0.100391,-0.186580,0.0,0.0,3
4,2023-01-01 00:00:04,0.0,0.142857,-0.444095,8.000000e-05,7.999999e-04,0.0,0.0,-0.32802,-0.369237,...,-0.767181,-0.180437,-0.507953,-0.716059,-0.774361,0.100393,-0.186548,0.0,0.0,4


In [10]:
device = torch.device('cpu')

In [11]:
train_df = cats_df[0:1000000]
train = train_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl',
       'bed1', 'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1',
       'cso1']]
test_df = cats_df[1000000:]


In [12]:
class tkgngc_data_processing:
    def __init__(self, data, device, num_timestamps=20):
        self.data = data
        self.ordered_column_names = self.data
        self.train_list = self.data.values.tolist()
        self.time_series_tensor = torch.tensor(self.train_list,dtype=torch.float32)
        # Entity and Relation indices
        self.entity_indices = torch.arange(len(self.train_list), dtype=torch.long)
        self.relation_indices = torch.tensor(
            [0 if i % 2 == 0 else 1 for i in range(len(self.entity_indices))],dtype=torch.long
        )

        self.timestamps = entity_indices
        # Timestamp binning
        self.num_timestamps = num_timestamps
        min_time, max_time = min(self.entity_indices),max(self.entity_indices)
        bins = torch.linspace(min_time,max_time,self.num_timestamps+1)
        self.timestamp_indices = torch.tensor(torch.bucketize(self.entity_indices,bins),dtype=torch.long) - 1
        self.timestamp_indices = torch.clamp(self.timestamp_indices, min = 0, max= num_timestamps -1)

        # Edge index

        self.edge_index = torch.tensor(
            [[i,i+1] for i in range(len(self.entity_indices) - 1)],dtype=torch.float32).t()



    
        

In [13]:
ordered_column_names = train.columns
train_list = train.values.tolist()
time_series_tensor = torch.tensor(train_list,dtype=torch.float32)

In [14]:
entity_indices = torch.arange(len(train_list), dtype=torch.long)
relation_indices = torch.tensor(
    [0 if i % 2 == 0 else 1 for i in range(len(entity_indices))],dtype=torch.long
)

In [15]:
timestamps = entity_indices

In [16]:
num_timestamps = 20
min_time, max_time = min(entity_indices),max(entity_indices)
bins = torch.linspace(min_time,max_time,num_timestamps+1)
timestamp_indices = torch.tensor(torch.bucketize(entity_indices,bins),dtype=torch.long) - 1
timestamp_indices = torch.clamp(timestamp_indices, min = 0, max= num_timestamps -1)

In [17]:
edge_index = torch.tensor(
    [[i,i+1] for i in range(len(entity_indices) - 1)],dtype=torch.float32).t()

In [18]:
time_series_tensor = time_series_tensor.to(device)
entity_indices=entity_indices.to(device)
relation_indices=relation_indices.to(device)
timestamp_indices=timestamp_indices.to(device)
edge_index=edge_index.to(device)

In [19]:
pretrained_tkg = PretrainedTKGEmbeddingWithTimestamps(
    num_entities=int(entity_indices.max().item()+1),
    num_relations=int(relation_indices.max().item()+1),
    embedding_dim=16,
    num_timestamps=num_timestamps,
).to(device)

In [20]:
quads = (
    entity_indices[:-1],  # Head entities
    relation_indices[:-1],  # Relations
    entity_indices[1:],  # Tail entities (shifted example)
    timestamp_indices[:-1],  # Timestamps
)

In [21]:
timestamp_indices[:-1]

tensor([ 0,  0,  0,  ..., 19, 19, 19])

In [22]:
relation_indices[:-1].shape

torch.Size([999999])

In [23]:
entity_indices[1:].shape

torch.Size([999999])

In [24]:
timestamp_indices[:-1].shape

torch.Size([999999])

In [25]:
pretrained_tkg.pretrain(quads, learning_rate=0.01, epochs=500)


Epoch 0, Loss: 3.465754985809326
Epoch 10, Loss: 2.829068899154663
Epoch 20, Loss: 2.30806827545166
Epoch 30, Loss: 1.8883682489395142
Epoch 40, Loss: 1.5519745349884033
Epoch 50, Loss: 1.2818219661712646
Epoch 60, Loss: 1.0636770725250244
Epoch 70, Loss: 0.8864285945892334
Epoch 80, Loss: 0.7416052222251892
Epoch 90, Loss: 0.622694194316864
Epoch 100, Loss: 0.5246222615242004
Epoch 110, Loss: 0.44341257214546204
Epoch 120, Loss: 0.3759232461452484
Epoch 130, Loss: 0.3196536600589752
Epoch 140, Loss: 0.2725988030433655
Epoch 150, Loss: 0.23314045369625092
Epoch 160, Loss: 0.19996574521064758
Epoch 170, Loss: 0.17200417816638947
Epoch 180, Loss: 0.14837954938411713
Epoch 190, Loss: 0.1283722221851349
Epoch 200, Loss: 0.11138947308063507
Epoch 210, Loss: 0.09694172441959381
Epoch 220, Loss: 0.0846235528588295
Epoch 230, Loss: 0.0740983784198761
Epoch 240, Loss: 0.06508605927228928
Epoch 250, Loss: 0.05735284462571144
Epoch 260, Loss: 0.0507032610476017
Epoch 270, Loss: 0.0449734069406986

In [26]:
for i, row in enumerate(cats_rows_list):
    potential_causes = metadata['root_cause'].unique().to_list()

    start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    anomaly = eval(row['affected'])[0]
    root_cause = row['root_cause']
    potential_causes.append(anomaly)
    mod_df = test_df[(test_df['timestamp']>= start_time) & (test_df['timestamp']<= end_time)]
    test = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl',
       'bed1', 'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1',
       'cso1']]
    break


In [27]:
test_data = tkgngc_data_processing(data=test, device=device, num_timestamps=20)

In [28]:
# Instantiate the full model
model = NGCWithPretrainedTKGAndTimestamps(
    pretrained_tkg=pretrained_tkg,
    input_dim=test_data.time_series_tensor.shape[1],
    hidden_dim=64,
    output_dim=1,
    confounder_latent_dim=8,
    use_sliding_window=False,
    window_size=10,
    step_size=2,
    regularization_type="l1",
    regularization_strength=0.01,
)

In [29]:
entity_emb, relation_emb, _, timestamp_emb = pretrained_tkg(
    test_data.entity_indices, test_data.relation_indices, test_data.entity_indices, test_data.timestamp_indices
)



In [30]:
entity_emb.shape

torch.Size([501, 16])

In [31]:
relation_emb.shape

torch.Size([501, 16])

In [32]:
timestamp_emb.shape

torch.Size([501, 16])

In [33]:
# Forward pass with the processed data
causality_scores, z, mean, log_var, x_reconstructed, reg_term = model(
    entity_indices=test_data.entity_indices,
    relation_indices=test_data.relation_indices,
    time_series_data=test_data.time_series_tensor,
    edge_index=test_data.edge_index,
    timestamp_indices=test_data.timestamp_indices,
)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (501x49 and 33x256)

In [ ]:

# Forward pass with the processed data
causality_scores, z, mean, log_var, x_reconstructed, reg_term = model(
    entity_indices=entity_indices,
    relation_indices=relation_indices,
    time_series_data=time_series_data,
    edge_index=edge_index,
    timestamp_indices=timestamp_indices,
)

# Output results
print("Causality Scores:", causality_scores)
print("Latent Confounder Representation:", z)
print("Reconstructed Time Series:", x_reconstructed)
print("Regularizati